<a href="https://colab.research.google.com/github/webb-e/S2_Landsat_Comparison/blob/main/S2_lake_wise_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script extracts the water area within each lake objectfrom the weekly Sentinle-2 water product. It is set up to run seperately for each region.

Written by Elizabeth Webb; last updated Aug 2024

In [ ]:
# Authenticate private account (only required for exporting to drive/gee/gcp)
from google.colab import auth
auth.authenticate_user()

# Earth Engine setup
import ee # Trigger the authentication flow.
ee.Authenticate()
ee.Initialize(project=" ") # Initialize the library.

# Google Drive setup
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Define variables for each region

In [ ]:
lakeshp = None
region_name = None
region_label = None
roi = None
def regionfun(region):
  global region_name, roi, lakeshp, region_label
  if region == 'TUK':
    lakeshp = ee.FeatureCollection("projects/alpod-412314/assets/Lake_extractions/TUK_extraction")
    region_name = 'TUK_MRD'
    roi = ee.FeatureCollection("projects/ee-webbe/assets/ROIs/TUK_roi")
    region_label = 'TUK'
  elif region == 'AND':
    lakeshp = ee.FeatureCollection("projects/alpod-412314/assets/Lake_extractions/AND_extraction")
    region_name = 'TUK_MRD'
    roi = ee.FeatureCollection("projects/ee-webbe/assets/ROIs/anderson_plain_roi")
    region_label = 'AND'
  elif region == 'MRD':
    lakeshp = ee.FeatureCollection("projects/alpod-412314/assets/Lake_extractions/MRD_extraction")
    region_name = 'TUK_MRD'
    roi = ee.FeatureCollection("projects/ee-webbe/assets/ROIs/MRD_roi")
    region_label = 'MRD'
  elif region == 'AKCP':
    lakeshp = ee.FeatureCollection("projects/alpod-412314/assets/Lake_extractions/AKCP_extraction")
    region_name = 'AKCP'
    roi = ee.FeatureCollection("projects/ee-webbe/assets/ROIs/coastalplain_roi")
    region_label = 'AKCP'
  elif region == 'YKD':
    lakeshp = ee.FeatureCollection("projects/alpod-412314/assets/Lake_extractions/YKD_extraction")
    region_name = 'YKD'
    roi = ee.FeatureCollection("projects/ee-webbe/assets/ROIs/YKdelta_roi")
    region_label = 'YKD'
  elif region == 'YKF':
    lakeshp = ee.FeatureCollection("projects/alpod-412314/assets/Lake_extractions/YKF_extraction")
    region_name = 'YKF'
    roi = ee.FeatureCollection("projects/ee-webbe/assets/ROIs/YKflats_roi")
    region_label = 'YKF'
  else:
    print("Invalid region")

Specify region and extract lake areas

In [ ]:
regionfun('YKD') # options = TUK, MRD, AND, AKCP, YKD, YKF

assetList = ee.data.listAssets('projects/alpod-412314/assets/region_weekly')['assets']
imgs = ee.ImageCollection.fromImages([ee.Image(asset['id']) for asset in assetList]).filter(ee.Filter.stringContains('system:id',region_name))#.filter(ee.Filter.stringContains('system:id','30'))
aoi = ee.Geometry.MultiPolygon(roi.geometry().getInfo()['coordinates'])


## ***PROPERTY EXTRACTION METHODS***

def summarizewater(image):
  ### prepare - get total area, masked area, and water area
  waterimg = image.select('water_occurance_max').gt(0) # create a binary image
  maskedimg = image.select('water_occurance_max').unmask(-1).eq(-1) # unmask and get binary image (masked/notmasked)
  totalimg = image.select('water_occurance_max').unmask(-1).gte(-1)  # get total number of pixels in region
  waterarea = waterimg.multiply(ee.Image.pixelArea()) # get area of water
  maskedarea = maskedimg.multiply(ee.Image.pixelArea()) #get area of masked
  totalarea = totalimg.multiply(ee.Image.pixelArea()) # total area
  areaImage = waterarea.addBands(maskedarea).addBands(totalarea)
  imgID = ee.String(image.get('system:id')).split('/').get(4)
  if region_name == 'TUK_MRD':
    week = ee.Number(ee.String(imgID).split('_').get(3))
    year = ee.Number(ee.String(imgID).split('_').get(2))
  else:
    week = ee.Number(ee.String(imgID).split('_').get(2))
    year = ee.Number(ee.String(imgID).split('_').get(1))
  ### take lake area per lake
  reduced = areaImage.reduceRegions(
              reducer=ee.Reducer.sum().unweighted(),
              collection=lakeshp,
              scale=10)
  def dealwithoutput(f):
    waterarea = ee.Number(f.get('water_occurance_max'))#.toFloat()#.divide(1e6)
    maskedarea = ee.Number(f.get('water_occurance_max_1'))#.divide(1e6).round()
    totalarea = ee.Number(f.get('water_occurance_max_2'))#.divide(1e6).round()
    unmaskedarea = totalarea.subtract(maskedarea)
    permasked = maskedarea.divide(totalarea).multiply(100).round()
    return f.set({ "S2_water_m2": waterarea, "percent_masked":permasked,'week': week, 'year': year })
  ##add feature properties to  output
  reduceroutput = reduced.map(dealwithoutput)
  ## only return lakes where nothing is masked
  answer = reduceroutput.filter(ee.Filter.eq('percent_masked', 0))
  return answer

summarized_lakes = imgs.map(summarizewater)
lake_summary = ee.FeatureCollection(summarized_lakes).flatten()
#print(lake_summary.first().getInfo())
description = str(region_label) + '_lake_areas'

task = ee.batch.Export.table.toDrive(**{
        'collection': ee.FeatureCollection(lake_summary),
        'description': description,
        'fileFormat': 'CSV',
        'folder': 'Lakewise_csvs',
          'selectors': ['lake_id', 'S2_water_m2', 'area_km2', 'perim_km', 'n_lakes', 'region', 'week', 'year']
  })
task.start()